In [ ]:
import time
import subprocess
import os
import torch
import cv2

# Manually set these variables based on your current setup
current_device = "<YOUR_DEVICE_NAME>"  # Change this to other hardware or GPU names as needed.
current_power_mode = "<YOUR_POWER_MODE>"  # Set the current power mode (just as metadata).

device = "GPU" if torch.cuda.is_available() else "CPU"
print(f"Currently using: {device}")

# Set the path to the YOLOv7 cloned GitHub directory
yolov7_directory = r"<PATH_TO_YOLOV7_DIRECTORY>"

# YOLOv7 weights paths
yolo_variants = {
    'Yolov7': r"<PATH_TO_YOLOV7_WEIGHTS>",
    'Yolov7-tiny': r"<PATH_TO_YOLOV7_TINY_WEIGHTS>",
    'Yolov7x': r"<PATH_TO_YOLOV7X_WEIGHTS>"
}

# Video paths
video_paths = [
    r"<PATH_TO_VIDEO_1>",
    r"<PATH_TO_VIDEO_2>",
    r"<PATH_TO_VIDEO_3>"
]

# Image sizes
image_sizes = [480, 640]

# Prepare results storage
results = []
results.append(f"Device: {current_device}\n")
results.append(f"Power Mode: {current_power_mode}\n")

# Loop over each video
for video_path in video_paths:
    cap = cv2.VideoCapture(video_path)
    video_length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    results.append(f"\n=== Video: {video_path} ===")
    results.append(f"Length: {video_length} frames")

    for variant, weights_path in yolo_variants.items():
        for img_size in image_sizes:
            # Run YOLOv7 detect.py for the entire video
            start_time = time.time()
            process = subprocess.run(['python', os.path.join(yolov7_directory, 'detect.py'),
                                      '--weights', weights_path,
                                      '--img', str(img_size),
                                      '--conf', '0.4',
                                      '--source', video_path,
                                      '--device', '0'],
                                     stdout=subprocess.PIPE,
                                     stderr=subprocess.PIPE)
            end_time = time.time()

            # Check for errors or outputs from detect.py
            print("STDOUT:", process.stdout.decode())
            print("STDERR:", process.stderr.decode())

            # Calculate the average FPS
            total_inference_time = end_time - start_time
            avg_fps = video_length / total_inference_time if total_inference_time > 0 else 0
            result_line = f"----- {variant} | Size {img_size} -----Average FPS: {avg_fps:.2f}"
            results.append(result_line)
            print(result_line)

# Save the results to a file
results_path = r"<PATH_TO_RESULTS_DIRECTORY>\yolov7_fps_results_test.txt"
with open(results_path, 'w') as f:
    for line in results:
        f.write(line + '\n')

print("FPS results written to", results_path)
